# Labour dataset

- use a simple base forecaster, the same formulation as the one in fable
- data sourced from [here](https://github.com/Nixtla/hierarchicalforecast/) (thanks!)

In [1]:
# :)
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [2]:
from datasetsforecast.hierarchical import HierarchicalData, HierarchicalInfo

from sktime.forecasting.sarimax import SARIMAX
from sktime.transformations.hierarchical.aggregate import Aggregator
from sktime.forecasting.reconcile import ReconcilerForecaster
from sktime.forecasting.model_selection import temporal_train_test_split

import pandas as pd
import numpy as np

The next few cells are also code from [here](https://github.com/Nixtla/hierarchicalforecast/) (thanks x3)

In [3]:
group = 'Labour'
init_cols, hier_cols, sep = ['Employment', 'Gender', 'Region'], ['Region', 'Employment', 'Gender'], ','
y, S, tags = HierarchicalData.load('data', group)
n_series = y['unique_id'].nunique()
n_series

57

* some info on the ts

In [4]:
meta_info_group = HierarchicalInfo[group]
h = meta_info_group.horizon
freq = meta_info_group.freq
sp = meta_info_group.seasonality

(h, freq, sp)

(8, 'MS', 12)

*  remove the aggregated levels

In [5]:
y = y.query('unique_id in @S.columns')
y[init_cols] = y['unique_id'].str.split(sep, expand=True)
y['ds'] = pd.PeriodIndex(y['ds'], freq="M")
y = y.set_index(hier_cols+['ds'])[['y']]

y


y
Region                           Employment            Gender     ds                 
 'Australian Capital Territory'] ['Employed full-time'  'Females' 1978-02   28.743290
                                                                  1978-03   26.649367
                                                                  1978-04   25.784798
                                                                  1978-05   27.543975
                                                                  1978-06   27.784657
...                                                                               ...
 'Western Australia']            ['Employed part-time'  'Males'   2019-08  133.887697
                                                                  2019-09  125.901203
                                                                  2019-10  133.986953
                                                                  2019-11  133.947213
                                                                  2019-12  134.159341

[16096 rows x 1 columns]

* remove unwanted characters from index

In [6]:
for i in range(y.index.nlevels-1):
    y[i] = (
        y.index.get_level_values(i)
        .str.replace("\['", "", regex=True)
        .str.replace("'\]", "", regex=True)
        .str.replace("'", "", regex=True)
    )

y = y.rename(columns={0: "Region", 1: "Employment", 2: "Gender"})
y = (
    y
    .reset_index('ds')
    .set_index(["Region", "Employment", "Gender", "ds"])
    .sort_index()
)
y

y
Region                        Employment         Gender   ds                 
 Australian Capital Territory Employed full-time  Females 1978-02   28.743290
                                                          1978-03   26.649367
                                                          1978-04   25.784798
                                                          1978-05   27.543975
                                                          1978-06   27.784657
...                                                                       ...
 Western Australia            Employed part-time  Males   2019-08  133.887697
                                                          2019-09  125.901203
                                                          2019-10  133.986953
                                                          2019-11  133.947213
                                                          2019-12  134.159341

[16096 rows x 1 columns]

Save data for fable

In [7]:
y.to_csv("./data/hierarchical/labour_bottomlevels.csv")
len(y.index.droplevel('ds').unique())

32

* now aggregate for totals

In [8]:
agg = Aggregator()
y = agg.fit_transform(y)
y

y
Region                        Employment         Gender   ds                   
 Australian Capital Territory Employed full-time  Females 1978-02     28.743290
                                                          1978-03     26.649367
                                                          1978-04     25.784798
                                                          1978-05     27.543975
                                                          1978-06     27.784657
...                                                                         ...
__total                       __total            __total  2019-08  12858.337240
                                                          2019-09  12925.936198
                                                          2019-10  12898.478058
                                                          2019-11  12992.526833
                                                          2019-12  13093.242657

[28671 rows x 1 columns]

Quick check on hierarchy with aggregated levels

* all good!

In [9]:
len(y.index.droplevel('ds').unique()) == n_series

True

Train test split

* to be the same as [here](https://github.com/Nixtla/hierarchicalforecast/) I've left out the last 8 obs

In [10]:
y_train, y_test = temporal_train_test_split(y, test_size=8)
y_train

y
Region                        Employment         Gender   ds                   
 Australian Capital Territory Employed full-time  Females 1978-02     28.743290
                                                          1978-03     26.649367
                                                          1978-04     25.784798
                                                          1978-05     27.543975
                                                          1978-06     27.784657
...                                                                         ...
__total                       __total            __total  2018-12  12839.166618
                                                          2019-01  12603.075613
                                                          2019-02  12802.157755
                                                          2019-03  12785.249118
                                                          2019-04  12831.683426

[28215 rows x 1 columns]

Fit the base forecasts

* use a simple AR1 model
* no intercept seems to perform better

In [11]:

base_forecaster = SARIMAX(order=(1,0,0), trend="n", enforce_stationarity=False)
base_forecaster.fit(y_train)
prds = base_forecaster.predict(fh=np.arange(1, h + 1)).rename(columns={'y': 'base'})
prds

base
Region                        Employment         Gender   ds                   
 Australian Capital Territory Employed full-time  Females 2019-05     70.487332
                                                          2019-06     70.569944
                                                          2019-07     70.652653
                                                          2019-08     70.735459
                                                          2019-09     70.818362
...                                                                         ...
__total                       __total            __total  2019-08  12909.480694
                                                          2019-09  12929.003599
                                                          2019-10  12948.556028
                                                          2019-11  12968.138027
                                                          2019-12  12987.749639

[456 rows x 1 columns]

Now fit the hierrachical reconciler forecasters

* note that each loop the base forecasters will be retrained
* we could do this a bit quicker for some reconciliation methods (Foecaster * Reconciler), but this is the easiest way

In [12]:
methods = sorted(ReconcilerForecaster.METHOD_LIST)

for method in methods:
    print(method)
    reconciler = ReconcilerForecaster(forecaster=base_forecaster, method=method)
    prds_recon = reconciler.fit_predict(y=y_train, fh=np.arange(1, h + 1)).rename(columns={'y': method})
    prds = pd.concat([prds, prds_recon], axis=1)

prds

bu
mint_cov
mint_shrink
ols
td_fcst
wls_str
wls_var


base  \
Region                        Employment         Gender   ds                      
 Australian Capital Territory Employed full-time  Females 2019-05     70.487332   
                                                          2019-06     70.569944   
                                                          2019-07     70.652653   
                                                          2019-08     70.735459   
                                                          2019-09     70.818362   
...                                                                         ...   
__total                       __total            __total  2019-08  12909.480694   
                                                          2019-09  12929.003599   
                                                          2019-10  12948.556028   
                                                          2019-11  12968.138027   
                                                          2019-12  12987.749639   

                                                                             bu  \
Region                        Employment         Gender   ds                      
 Australian Capital Territory Employed full-time  Females 2019-05     70.487332   
                                                          2019-06     70.569944   
                                                          2019-07     70.652653   
                                                          2019-08     70.735459   
                                                          2019-09     70.818362   
...                                                                         ...   
__total                       __total            __total  2019-08  12899.606564   
                                                          2019-09  12916.660189   
                                                          2019-10  12933.743045   
                                                          2019-11  12950.855181   
                                                          2019-12  12967.996644   

                                                                       mint_cov  \
Region                        Employment         Gender   ds                      
 Australian Capital Territory Employed full-time  Females 2019-05     70.533325   
                                                          2019-06     70.661959   
                                                          2019-07     70.790722   
                                                          2019-08     70.919616   
                                                          2019-09     71.048639   
...                                                                         ...   
__total                       __total            __total  2019-08  12908.851977   
                                                          2019-09  12928.217907   
                                                          2019-10  12947.613445   
                                                          2019-11  12967.038638   
                                                          2019-12  12986.493529   

                                                                    mint_shrink  \
Region                        Employment         Gender   ds                      
 Australian Capital Territory Employed full-time  Females 2019-05     70.636396   
                                                          2019-06     70.868021   
                                                          2019-07     71.099691   
                                                          2019-08     71.331405   
                                                          2019-09     71.563161   
...                                                                         ...   
__total                       __total            __total  2019-08  12909.462296   
                                                          2019-09  12928.980622   
     

Calculate the RMSE

* this is much the same as the fable package
* there are some minor diffences between the methods based on the residual covariance matrix

In [13]:
eval = pd.concat([prds, y_test], axis=1)
eval = eval.melt(id_vars='y', ignore_index=False)

eval.groupby('variable').apply(
    lambda x: np.round((((x['y'] - x['value'])**2).mean())**(1/2), 3)
)

variable
base           17.523
bu             17.819
mint_cov       17.519
mint_shrink    17.580
ols            17.519
td_fcst        17.466
wls_str        17.505
wls_var        17.573
dtype: float64